<a href="https://colab.research.google.com/github/bhnunes/CNNStudy/blob/main/CNNStudy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import json
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import img_to_array, load_img
from keras.applications.imagenet_utils import preprocess_input
import requests
from collections import Counter, defaultdict
import os
import pickle as pk

In [2]:
# URL of the raw file on GitHub
url = "https://raw.githubusercontent.com/bhnunes/CNNStudy/main/imagenet_class_index.json"
testUrl = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/d0/German_Shepherd_-_DSC_0346_%2810096362833%29.jpg/220px-German_Shepherd_-_DSC_0346_%2810096362833%29.jpg"


def getFile(url,fileName, mode):
  response = requests.get(url)
  if response.status_code ==  200:
      with open(fileName, mode) as file:
        if mode =='w':
          file.write(response.text)
        else:
          file.write(response.content)
  else:
      print(f"Failed to download the file. Status code: {response.status_code}")





In [3]:
getFile(url,'imagenet_class_index.json', 'w')
getFile(testUrl,'test_image.jpg', 'wb')


In [4]:
#download VGG16
vgg16=VGG16(weights='imagenet')

553467096/553467096 [==============================] - 7s 0us/step


In [5]:
vgg16.save('vgg16.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
def prepare_image(img_path):
  img=load_img(img_path, target_size=(224,224))
  x=img_to_array(img)
  x=np.expand_dims(x,axis=0)
  x=preprocess_input(x)
  return x

In [7]:
imagePath = "/content/test_image.jpg"
y = prepare_image(imagePath)

In [8]:
preds=vgg16.predict(y)

1/1 [==============================] - 1s 1s/step


In [9]:
CLASS_INDEX=None
CLASS_INDEX_PATH="/content/imagenet_class_index.json"

In [10]:
def get_predictions(preds,top=5):
  global CLASS_INDEX

  CLASS_INDEX=json.load(open(CLASS_INDEX_PATH))

  results=[]
  for pred in preds:
    top_indices=pred.argsort()[-top:][::-1]
    result=[tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
    result.sort(key=lambda x: x[2], reverse=True)
    results.append(result)
  return results

In [11]:
print(get_predictions(preds,top=5))

[[('n02106662', 'German_shepherd', 0.99837816), ('n02105162', 'malinois', 0.0015710895), ('n03803284', 'muzzle', 1.9646503e-05), ('n02105056', 'groenendael', 9.4065235e-06), ('n02105412', 'kelpie', 7.3132423e-06)]]


In [12]:
def get_car_categories(FolderPath):
    d = defaultdict(float)
    img_list = os.listdir(FolderPath)
    for i, img_path in enumerate(img_list):
        img = prepare_image(FolderPath+'/'+img_path)
        out = vgg16.predict(img)
        top = get_predictions(out, top=5)
        for j in top[0]:
            d[j[0:2]] += j[2]
        if i % 100 == 0:
            print (i, '/', len(img_list), 'complete')
    return Counter(d)

In [13]:
cat_counter=get_car_categories('/content/drive/MyDrive/CNN_Training/Car_Image_damagevswhole/data1a/training/01-whole')

1/1 [==============================] - 1s 975ms/step
0 / 920 complete
1/1 [==============================] - 1s 525ms/step
100 / 920 complete
1/1 [==============================] - 1s 869ms/step
200 / 920 complete
1/1 [==============================] - 1s 502ms/step
300 / 920 complete
1/1 [==============================] - 1s 507ms/step
400 / 920 complete
1/1 [==============================] - 0s 499ms/step
500 / 920 complete
1/1 [==============================] - 1s 853ms/step
600 / 920 complete
1/1 [==============================] - 1s 522ms/step
700 / 920 complete
1/1 [==============================] - 1s 510ms/step
800 / 920 complete
1/1 [==============================] - 1s 862ms/step
900 / 920 complete
1/1 [==============================] - 1s 509ms/step


In [14]:
with open('/content/cat_counter.pk', 'wb') as f:
  pk.dump(cat_counter,f,-1)

In [ ]:
# This piece of code will be used to load already saved cat_counter
#with open('/content/cat_counter.pk', 'rb') as f:
  #cat_counter = pk.load(f)

In [16]:
cat_list  = [k for k, v in cat_counter.most_common()[:27]]

# First Check - car or not

In [20]:
def car_categories_gate(image_path, cat_list):
    img = prepare_image(image_path)
    out = vgg16.predict(img)
    top = get_predictions(out, top=5)
    #print ("Validating that this is a picture of your car...")
    for j in top[0]:
        print(j)
        if j[0:2] in cat_list:
            print (j[0:2])
            return "Validation complete - proceed to damage evaluation"
    return "Are you sure this is a picture of your car? Please take another picture (try a different angle or lighting) and try again."

In [22]:
car_categories_gate('/content/test_car.jpg', cat_list)

1/1 [==============================] - 1s 503ms/step
('n03770679', 'minivan', 0.4084556)
('n03770679', 'minivan')


'Validation complete - proceed to damage evaluation'